In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(5.0,4.0) #set default size of plot
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
np.random.seed(1)

In [2]:
def sigmoid(Z):
    A=1/(1+np.exp(-Z)) #sigmoid eqn
    cache=Z
    
    return A,cache

def relu(Z):
    A=np.maximum(0,Z)
    cache=Z
    return A,cache

def relu_backward(dA,cache):
    Z=cache
    dZ=np.array(dA,copy=True) #dA -differntiate of A
    dZ[Z<=0]=0
    return dZ

def sigmoid_backward(dA,cache):
    Z=cache
    s=1/(1+np.exp(-z))
    dZ=dA*s*(1-s) #differntiate of sigmoid
    
    return dZ

##### Steps for building L layer of Neural Network in Deep Learning

- Initialize parameter  (W1,b1,W2,b2)
- Forward propogation
- Find cost fun
- Backward propogation

#### 1 .Initialization

- The model's Structure is: LINEAR->RELU->LINEAR->SIGMOID
- use this random initialization for the weight matrices:np.random.randn(shape)*0.01 with the correct shape
- use Zero initialization for the biases:np.zeros(shape)


In [3]:
def initialize_parameter(n_x,n_h,n_y):
    w1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros([n_h,1])
    w2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros([n_y,1])
    
    parameter={'w1':w1,
              'b1':b1,
              'w2':w2,
              'b2':b2}
    
    return parameter
# initialize_parameter(4,2,3)

In [4]:
parameter=initialize_parameter(3,2,1) #fst layer,2nd layer,3rd layer

print('w1 ='+str(parameter['w1']))
print('b1 ='+str(parameter['b1']))
print('w2 ='+str(parameter['w2']))
print('b2 ='+str(parameter['b2']))

w1 =[[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 =[[0.]
 [0.]]
w2 =[[ 0.01744812 -0.00761207]]
b2 =[[0.]]


##### parameter initialization for L layer neural network

In [5]:
def initilaize_parameter_deep(layer_dims):
    
    np.random.seed(3)
    parameter={}
    L=len(layer_dims)
    
    for i in range(1,L):
        parameter['w'+str(i)]=np.random.randn(layer_dims[i],layer_dims[i-1])*0.01
        parameter['b'+str(i)]=np.zeros([layer_dims[i],1])
        
    return parameter

In [6]:
parameter=initilaize_parameter_deep([5,4,3]) #fst layer,2nd layer,3rd layer

print('w1 ='+str(parameter['w1']))
print('b1 ='+str(parameter['b1']))
print('w2 ='+str(parameter['w2']))
print('b2 ='+str(parameter['b2']))

w1 =[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 =[[0.]
 [0.]
 [0.]
 [0.]]
w2 =[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 =[[0.]
 [0.]
 [0.]]


#### 2 Forward propogation



In [7]:
def linear_forward(A,w,b):
    Z=np.dot(w,A)+b
    cache=(A,w,b)
    return Z,cache

- linear action forward

In [8]:
def linear_activation_forward(A_prev,w,b,activation):
    if activation=='sigmoid':
        Z,linear_cache=linear_forward(A_prev,w,b)
        A,activation_cache=sigmoid(Z)
        
    elif activation=='relu':
        Z,linear_cache=linear_forward(A_prev,w,b)
        A,activation_cache=relu(Z)
        
    cache=(linear_cache,activation_cache)
    
    return A,cache

In [9]:
def L_model_forward(X,parameter):
    caches=[]
    A=X
    L=len(parameter)//2
    
    for i in range(1,L):
        A_prev=A
        
        A,cache=linear_activation_forward(A_prev,
                                         parameter['w'+str(i)],
                                         parameter['b'+str(i)],
                                         activation='relu')
        
        cache.append(cache)
        
    AL,cache=linear_activation_forward(A,
                                      parameter['w'+str(i)],
                                      parameter['b'+str(i)],
                                      activation='sigmoid')
    
    cache.append(cache)
    
    return AL,cache

#### compute the cross-entropy cost ...cost function

In [10]:
def compute_cost(AL,Y):
    m=Y.shape[1]
    cost=-1/m*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost=np.squeeze(cost)
    
    return cost


### 3.Backward propogation

- Z[l]=W[l]A[l-1]+b[l]



In [13]:
def linear_backward(dZ,cache):
    A_prev,w,b=cache
    m=A_prev.shape[1]
    
    dw=1/m*np.dot(dZ,A_prev.T)
    db=1/m*np.sum(dZ,axis=1,keepdims=True)
    dA_prev=np.dot(w.T,dZ)
    
    return dA_prev,dw,db
    

- linear activation backward

In [14]:
def linear_activation_backward(dA,cache,activation):
    linear_cache,activation_cache=cache
    
    if activation=='relu':
        dZ=relu_backward(dA,activation_cache)
        dA_prev,dw,db=linear_backward(dZ,linear_cache)
        
    elif activation=='sigmoid':
        dZ=sigmoid_backward(dA,activation_cache)
        dA_prev.dw.db=linear_backward(dZ,linear_cache)
        
    return dA_prev,dw,db

- L model backward

In [15]:
def L_model_backward(AL,Y,cache):
    '[LINEAR->RELU]*(L-1)->LINEAR->SIGMOID'
    
    grads={}
    L=len(cache) #calculate numb of layers
    m=AL.shape[1]
    Y=Y.reshape(AL.shape)
    
    dAL=-(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache=cache[L-1]
    grads['dA'+str(L-1)],grads['dw'+str(L)],grads['db'+str(L)]=\
    linear_backward(sigmoid_backward(dAL,current_cache[1]),current_cache[0])
    
    for i in reversed(range(L-1)):
        current_cache=cache[1]
        dA_prev_temp,dw_temp,db_temp=linear_activation_backward(grads['dA'+str(i+1)],current_cache,activation='relu')
        
        grads['dA'+str(i)]=dA_prev_temp
        grads['dw'+str(i+1)]=dw_temp
        grads['db'+str(i)]=db_temp
        
    return grads

In [16]:
for i in reversed(range(5)):
    print(i)

4
3
2
1
0


#### 4. update parameters

In [17]:
def update_parameter(params,grads,learning_rate):
    parameter=params.copy()
    L=len(parameter)//2
    
    for i in range(L):
        parameter['w'+str(i+1)]=parameter['w'+str(i+1)]-learning_rate*grads['dw'+str(i+1)]
        parameter['b'+str(i+1)]=parameter['b'+str(i+1)]-learning_rate*grads['db'+str(i+1)]
    return parameter